In [1]:
import json 
import pandas as pd
import csv 
import pickle
import numpy as np
import re
import os
import gzip
from tqdm import tqdm

In [2]:
cta_vocab = open("./cta_vocab.txt", 'r')
cta_vocab = [line.replace('\n', '').split('\t')[1] for line in cta_vocab.readlines()]  
len(cta_vocab) # 255

255

In [3]:
def output_cta_label_ids(label_list):
    y = [0] * len(cta_vocab)
    for i in label_list:
        y[cta_vocab.index(i)] = 1
    return y

In [4]:
df_ori_cta_train = pd.read_json('Output/train.table_col_type.json', orient='records')
df_ori_cta_train
df_ori_cta_dev = pd.read_json('Output/dev.table_col_type.json', orient='records')
df_ori_cta_dev
df_ori_cta_test = pd.read_json('Output/test.table_col_type.json', orient='records')
df_ori_cta_test

,0,1,2,3,4,5,6,7
0,27295818-29,2010–11 rangers f.c. season,27295818,overall,,"[competition, started round, final position / ...","[[[[0, 0], [26980923, 'Scottish Premier League...","[[time.event], [time.event], [time.event]]"
1,27298128-1,kagawa at-large district (house of councillors),27298128,elected councillors,,[election year],"[[[[0, 0], [31629558, '1947']], [[1, 0], [3162...",[[time.event]]
2,27298635-1,list of museums in northumberland,27298635,,see also,[town/city],"[[[[0, 0], [129195, 'Alnwick']], [[1, 0], [208...",[[location.location]]
3,27308318-1,list of museums in north yorkshire,27308318,,defunct museums,"[town/city, region]","[[[[0, 0], [2400278, 'Aldborough']], [[1, 0], ...","[[location.location], [location.location]]"
4,6675886-1,indy japan 300,6675886,past winners,usac championship car history (non-championsh...,"[season season, driver driver, team team, chas...","[[[[2, 0], [5710555, '1998']], [[3, 0], [64984...","[[sports.sports_league_season, time.event], [p..."
...,...,...,...,...,...,...,...,...
4759,8086435-1,112 discography,8086435,studio albums,"list of studio albums, with selected chart pos...",[title title],"[[[[0, 0], [2226709, '112']], [[1, 0], [533977...",[[music.album]]
4760,808980-1,list of rulers of austria,808980,house of babenberg,,"[name, marriages, death]","[[[[0, 0], [1100652, 'Leopold I the Illustriou...","[[people.person, people.family_member, royalty..."
4761,808980-2,list of rulers of austria,808980,house of babenberg,,"[name, marriages, death]","[[[[0, 0], [1116423, 'Henry II Jasomirgott (He...","[[people.person, people.family_member, royalty..."
4762,809525-4,nba conference finals,809525,division finals,,"[year, champion, runner-up]","[[[[0, 0], [8569886, '1949']], [[1, 0], [86170...","[[award.competition, time.event], [basketball...."


In [5]:
for idx, row in df_ori_cta_test.head(1).iterrows(): 
    print(row[6])
    print(row[7])

[[[[0, 0], [26980923, 'Scottish Premier League']], [[1, 0], [18255941, 'UEFA Champions League']], [[2, 0], [20795986, 'UEFA Europa League']], [[3, 0], [28233480, 'League Cup']], [[4, 0], [28233536, 'Scottish Cup']]], [[[1, 1], [18255941, 'Group stage']], [[2, 1], [20795986, 'Round of 32']], [[3, 1], [28233480, '3rd Round']], [[4, 1], [28233536, '4th Round']]], [[[1, 2], [18255941, 'Group stage']], [[2, 2], [20795986, 'Round of 16']], [[3, 2], [28233480, 'Winners']], [[4, 2], [28233536, '5th Round']]]]
[['time.event'], ['time.event'], ['time.event']]


In [6]:
def clean_text(text):
    # this join may cause problem
    if(isinstance(text, dict)):
        text = ' '.join([ clean_text(v) for k, v in text.items()] )
    elif(isinstance(text, list)):
        text = map(clean_text, text)
        text = ' '.join(text)
        
    if pd.isnull(text):
        return ''
    
    #Remove non-ASCII characters
    text = re.sub(r"[^\x00-\x7F]+", " ", str(text))
    
    #Remove excess whitespaces
    text = re.sub(' +', ' ', str(text)).strip()
    
    return text

def create_table(table_id, data, labels, split, csv_list):
    curr_dict = {}
    for i in data:
        for j in i:
            curr_ele = j[0]
            curr_col = curr_ele[1]
            curr_row = curr_ele[0]
            curr_val = j[1][1]

            if curr_dict.get(curr_col) == None: 
                curr_dict[curr_col] = []
                curr_dict[curr_col].append([curr_row, curr_val])
            else:
                curr_dict[curr_col].append([curr_row, curr_val])
                
    sorted_dict = {key: value for key, value in sorted(curr_dict.items())}
    
    new_df_ll = []
    for key in sorted_dict:
        curr_li = curr_dict.get(key)
        curr_row = [i[1] for i in curr_li]
        new_df_ll.append(curr_row)
    new_df = pd.DataFrame(new_df_ll)
    new_df = new_df.transpose()
    
    new_df = new_df.apply(np.vectorize(clean_text))
    
    counter = 0
    for i in labels:
        y = output_cta_label_ids(i)
        csv_list.append([table_id, counter, y])
        counter += 1
        
    save_to = '../turl/data/turl_cta_data/{}/'.format(split) + str(table_id) + '.json.gz'
    
    if os.path.exists(save_to):
        pass
    else:
        new_df.to_json(save_to, compression='gzip', orient='records', lines=True)
    return 1

def create_table_and_csv(df, split, csv_list):
    print('Handling {} rows'.format(len(df)))
    for idx, row in tqdm(df.iterrows()):
        create_table(row[0], row[6], row[7], split, csv_list)

In [10]:
train_final_csv_list = []
dev_final_csv_list = []
test_final_csv_list = []
# 
print('Start test')
create_table_and_csv(df_ori_cta_test, 'Test', test_final_csv_list)
print('Start dev')
create_table_and_csv(df_ori_cta_dev, 'Validation', dev_final_csv_list)
print('Start train')
create_table_and_csv(df_ori_cta_train, 'Train', train_final_csv_list)

test_df = pd.DataFrame(test_final_csv_list, columns=['table_id', 'column_index', 'label_ids'])
dev_df = pd.DataFrame(dev_final_csv_list, columns=['table_id', 'column_index', 'label_ids'])
train_df = pd.DataFrame(train_final_csv_list, columns=['table_id', 'column_index', 'label_ids'])

final_dict = {}
final_dict['train'] = train_df
final_dict['dev'] = dev_df
final_dict['test'] = test_df

f = open('../turl/data/turl_cta_data/cta_turl_lm.pkl','wb')
pickle.dump(final_dict,f)
f.close()

Start test
Handling 4764 rows


4764it [00:13, 366.03it/s]


Start dev
Handling 4844 rows


4844it [00:12, 377.53it/s]


Start train
Handling 397098 rows


397098it [14:33, 454.43it/s]


In [11]:
test_df

,table_id,column_index,label_ids
0,27295818-29,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,27295818-29,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,27295818-29,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,27298128-1,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,27298635-1,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...
13020,809525-4,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
13021,809525-4,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
13022,809525-4,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
13023,8097449-11,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [12]:
df = pd.read_json('../turl/data/turl_cta_data/Train/8099475-1.json.gz', compression='gzip', lines=True)
print(df)

                                      0                        1
0                          Ice Princess  Steven T. Murray (2008)
1                          The Preacher  Steven T. Murray (2009)
2                       The Stonecutter  Steven T. Murray (2010)
3  The Gallows Bird a.p.a. The Stranger  Steven T. Murray (2011)
4                      The Hidden Child    Tiina Nunnally (2011)
5                          The Drowning    Tiina Nunnally (2012)
6              Flavours from Fj llbacka                         
7                The Angel Maker's Wife                         
8                    Feast, Food & Love                         


In [13]:
with open('../turl/data/turl_cta_data/cta_turl_lm.pkl', 'rb') as f:
    dict = pd.read_pickle(f)
dict

{'train':           table_id  column_index  \
 0        2728176-1             0   
 1        2728185-1             0   
 2       27282030-1             0   
 3        2728219-1             0   
 4        2728231-1             0   
 ...            ...           ...   
 628249   8099475-1             1   
 628250   8099493-1             0   
 628251   8099493-1             1   
 628252   8099493-9             0   
 628253   8099493-9             1   
 
                                                 label_ids  
 0       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
 1       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
 2       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
 3       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
 4       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
 ...                                                   ...  
 628249  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
 628250  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..

In [14]:
dict.get('train').column_index.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])